In [1]:
%load_ext rpy2.ipython

In [80]:
%%R -o df_ck

# lib_ = "/home/ferrari/R/x86_64-redhat-linux-gnu-library/3.5"

library(clusterProfiler)
library(org.Mm.eg.db)

deg = read.table("../DATA/DE_genes_withLFCthr1_mESCvsNPC48h.tsv", sep="\t", )
row.names(deg) = gsub("\\.[0-9]+","",row.names(deg))
head(deg, n=100)
deg_up = unique(row.names(deg[(deg$padj<0.01) & (deg$log2FoldChange>0),]))
deg_down = unique(row.names(deg[(deg$padj<0.01) & (deg$log2FoldChange<0),]))

universe_ = unique(row.names(deg))

### TRANSLATE ENSEMBLE TO ENTREZ AND SYMBOLS

up = bitr(deg_up, fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")
down = bitr(deg_down, fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")

universe = bitr(universe_, fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")

list_def = list(deg_up = up$ENTREZID, deg_down = down$ENTREZID)
# # list_def
ck <- compareCluster(geneCluster = list_def,
                     universe = universe$ENTREZID,
                     fun = "enrichGO",
                     OrgDb = "org.Mm.eg.db",
                     ont = "BP",
                     pAdjustMethod = "BH",
                     qvalueCutoff = 0.05,
                     pvalueCutoff = 0.05,
                     readable = TRUE)

df_ck = as.data.frame(ck)


In [84]:
%%R
ck_filt = gofilter(ck)
pdf("../FIGURES/GO_ENRICHMENT_NPC48h_vs_mESC.pdf", width=8, height=5)
dp = dotplot(ck_filt, showCategory=10)
print(dp)
dev.off()

png 
  2 
